In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')


In [9]:
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/Forestaciones_uru')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
# Capturar el polígono dibujado como ROI
roi = Map.user_roi
coords = roi.coordinates()

print(coords.getInfo())

geom = ee.Geometry.Polygon(coords)
roi = geom

[[[-58.295174, -31.823898], [-58.295174, -31.694872], [-58.072357, -31.694872], [-58.072357, -31.823898], [-58.295174, -31.823898]]]


In [13]:
# Capturar el polígono dibujado como ROI
# roi = Map.user_roi
# coords = roi.coordinates()

# print(coords.getInfo())

# geom = ee.Geometry.Polygon(coords)
# roi = geom

In [11]:
# ---------------------------------------------------------------------
# 1) PARAMETERS
# ---------------------------------------------------------------------
EMB_COLLECTION  = 'GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL'
GEDI_COLLECTION = 'LARSE/GEDI/GEDI04_A_002_MONTHLY'
TARGET_BAND     = 'agbd'   # above-ground biomass density (Mg ha⁻¹)


# ---------------------------------------------------------------------
# 3) YEAR 2: 2024
# ---------------------------------------------------------------------
YEAR = 2024
START = f'{YEAR}-01-01'
END = f'{YEAR}-12-31'

embedding = (ee.ImageCollection(EMB_COLLECTION)
              .filterDate(START, END))

embedding_image = embedding.median().clip(roi)
# ---------------------------------------------------------------------
# Add SRTM DEM to stack
# ---------------------------------------------------------------------
srtm = (ee.Image('USGS/SRTMGL1_003')
          .select('elevation')
          .clip(roi)
          .rename('elevation'))

dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(START, END)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

label = dw.rename("label")
# (opcional) derivados topográficos
terrain = ee.Terrain.products(srtm)
slope   = terrain.select('slope').rename('slope')
aspect  = terrain.select('aspect').rename('aspect')

# Stack final: embeddings + DEM (+ slope/aspect si querés)
embedding_image = embedding_image.addBands([srtm, slope, aspect,label])

In [12]:
embedding_image

In [13]:
Map.addLayer(embedding_image,{},"embedding")
Map.addLayer(embedding_image.select("elevation"),{},"elevation")

Map

Map(bottom=622238.0, center=[-31.761450771533056, -58.22530746459961], controls=(WidgetControl(options=['posit…

In [14]:
def mask_excluding_built(image, built_id=6):
    dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(START, END)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

    not_built_mask = dw.neq(built_id)  # <-- TODO menos built
    return image.updateMask(not_built_mask)


# Obtener los datos GEDI de la altura de dosel
def get_gedi_canopy_height(band_name):
    return (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
        .filterBounds(roi)
        .filterDate(START, END)
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)))
        .select(band_name)
        .median()
        .toFloat()
        .clip(roi)
    )

gedi_rh98 = get_gedi_canopy_height('rh98').rename('rh98')


dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(START, END)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

# excluir construido
gedi_masked = mask_excluding_built(gedi_rh98, built_id=6)
image = embedding_image.addBands(gedi_masked.rename('rh98'))
image = image.addBands(dw.rename('label'))

In [29]:
image

In [56]:
Map.addLayer(image.select('label'),{},"agbd")
Map

Map(bottom=79213.0, center=[-34.45448326886294, -57.3980712890625], controls=(WidgetControl(options=['position…

In [15]:
import time

sample = image.sample(
    scale=10,  # Ajusta la escala según sea necesario para tus datos
    region=roi,
    geometries=True  # Incluir geometría de los puntos
)

# Exportar la tabla de muestra a Google Drive en formato CSV
export_task = ee.batch.Export.table.toDrive(
    collection=sample,
    description='costa_de_uruguay_completo',
    folder='CH-delta',
    fileNamePrefix=f'costa_de_uruguay_completo',
    fileFormat='CSV'
)


# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'costa_de_uruguay_completo', 'priority': 100, 'creation_timestamp_ms': 1764012012689, 'update_timestamp_ms': 1764012040902, 'start_timestamp_ms': 1764012020701, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 59.972892761, 'id': 'KJH6T24UKLXSS6OMLOZHWUKN', 'name': 'projects/ee-facuboladerasgee/operations/KJH6T24UKLXSS6OMLOZHWUKN'}
Exportación en progreso...
Exportación completada con éxito.


# Predicts

In [18]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']

image = embedding_image.select(feature_names)

In [19]:
image

In [9]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']

label = "rh98"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/DELTA-CHM_ext2"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [19]:
visPredictedBiomass = {
    'min': 0,
    'max': 30,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}


Map.centerObject(roi)
Map.addLayer(
    classified,
    visPredictedBiomass,
    "im1",)


Map

Map(bottom=630472.0, center=[-34.1333725, -58.823547], controls=(WidgetControl(options=['position', 'transpare…

In [10]:
# Export 2017
task1 = ee.batch.Export.image.toDrive(
    image=classified,
    description='Prediccion_2024_colon',
    folder='CH-delta',       # Carpeta en tu Google Drive
    fileNamePrefix='Prediccion_2024_colon',
    region=roi,
    scale=10,                    # Resolución de 100 metros
    crs='EPSG:4326',              # Puedes ajustar el CRS según tus datos
    maxPixels=1e13
)
task1.start()

print("🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE")

EEException: Collection.loadTable: Collection asset 'users/facuboladerasgee/DELTA-CHM_ext2' not found.

In [34]:
image

# Cargar rf dividido

In [20]:
feature_names = ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'aspect', 'elevation', 'label', 'slope']



def load_rf_from_parts(base_asset_id, n_parts=27):
    """
    base_asset_id: 'users/facuboladerasgee/Rf_CHM-FORESTAL'
    n_parts: 27
    """
    # 1) cargar y mergear todas las parts
    merged = ee.FeatureCollection([])

    for i in range(n_parts):
        part_id = f"{base_asset_id}_part_{i}"
        fc_part = ee.FeatureCollection(part_id)
        merged = merged.merge(fc_part)

    # 2) reconstruir classifier (regresión)
    clf = ml.fc_to_classifier(merged).setOutputMode('REGRESSION')
    return clf


# ---- uso ----
base_asset_id = "users/facuboladerasgee/Rf_CHM-FORESTAL"
another_classifier = load_rf_from_parts(base_asset_id, n_parts=27)

classified = image.classify(another_classifier)

In [47]:
# pred = image.classify(another_classifier).rename("pred")
# # Desacoplar grafo grande
# pred_simplified = pred.reproject(pred.projection()).focal_mean(1)

In [46]:
# visPredictedBiomass = {
#     'min': 0,
#     'max': 30,
#     'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
# }


# Map.centerObject(roi)
# Map.addLayer(
#     pred_simplified,
#     visPredictedBiomass,
#     "im1",)

# Map

In [21]:
# Export 2017
task1 = ee.batch.Export.image.toDrive(
    image=classified,
    description='Prediccion_2024_costa',
    folder='CH-delta',       # Carpeta en tu Google Drive
    fileNamePrefix='Prediccion_2024_costa',
    region=roi,
    scale=10,                    # Resolución de 100 metros
    crs='EPSG:4326',              # Puedes ajustar el CRS según tus datos
    maxPixels=1e13
)
task1.start()

print("🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE")

🚀 Exportaciones lanzadas: revisa la pestaña 'Tasks' en GEE
